In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pyplot as plt



In [35]:
np.random.seed(100)

In [36]:
df = pd.read_csv(f'../data/market_integrates_100.csv')
print(df)


     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.433865         2.042325  21.721590   
1             1         2         5.907737         2.412709  18.586650   
2             1         3         5.943911         2.174390  20.086869   
3             1         4         5.329441         2.844997  20.126112   
4             1         5         5.083033         2.137079  22.903818   
..          ...       ...              ...              ...        ...   
995         100         6         5.360283         2.580602  24.693921   
996         100         7         5.958183         2.650134  22.406677   
997         100         8         5.184745         2.346844  19.703203   
998         100         9         5.223118         2.464959  20.661943   
999         100        10         5.019838         2.675947  23.255443   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0        14.701881  0.061575  0.4322

In [37]:
# theta_0 = np.array([2, -0.5, 0.3, 0.05, 1.])
theta_0 = np.array([2., -0.5, -0.3, 0.5, 1.])




In [38]:
def simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t): 
    beta = theta[:3]
    mu = theta[3]
    omega = theta[4]
    X0 = np.ones(n_firms)
    X1 = df.characteristic1.head(n_firms)
    X2 = df.characteristic2.head(n_firms)
    X_car = np.column_stack((X1, X2))
    X = np.column_stack((X0, X_car))

    X_for_utility = np.repeat(X, n_consumers, axis=0)
    price_r = np.reshape(np.array(df.predict_prices[t*n_firms:(t+1)*n_firms]), (1, n_firms))
    alpha_i = np.reshape(-(np.exp(mu + omega*v_p)), (n_consumers, 1))
    random_coeff = np.ravel((alpha_i*price_r).T)

    u = X_for_utility@beta + random_coeff
    u_r = np.reshape(u, (n_firms, n_consumers))
    sum_u = np.sum(np.exp(u_r))

    all_probs = np.exp(u_r)/(1 + sum_u)
    market_shares = np.sum(all_probs, axis=1)
    
    return market_shares

In [50]:
def get_indirect_utility(theta, df, n_consumers, n_firms, t):
    beta = theta[:3]
    mu = theta[3]
    omega = theta[4]
    X0 = np.ones(n_firms)
    X1 = df.characteristic1.head(n_firms)
    X2 = df.characteristic2.head(n_firms)
    X_car = np.column_stack((X1, X2))
    X = np.column_stack((X0, X_car))

    X_for_utility = np.repeat(X, n_consumers, axis=0)
    price_r = np.reshape(np.array(df.prices[t*n_firms:(t+1)*n_firms]), (1, n_firms))
    alpha = -(np.exp(mu + omega**2/2))
    random_coeff_mean = np.ravel((alpha*price_r).T)

    indirect_utility = X_for_utility@beta + random_coeff_mean
    return indirect_utility




In [40]:
def f(theta, df, n_consumers, n_firms, T):
    true_share = df.shares
    all_est_shares = np.zeros(T*n_firms)
    v_p_all = np.random.normal(0, 1, n_consumers*T)
    for t in range(0, T-1):
        v_p = v_p_all[t*n_consumers:(t+1)*n_consumers]
        estimates_shares_market = simulate_market_shares_per_period(theta, df, v_p, n_consumers, n_firms, t)
        all_est_shares[t*n_firms:(t+1)*n_firms] = estimates_shares_market
    return np.linalg.norm(true_share - all_est_shares)

In [51]:
indirect_utilities_true_par = get_indirect_utility(theta_0, df, n_consumers=500, n_firms=10, t=0)
print(indirect_utilities_true_par)

ValueError: operands could not be broadcast together with shapes (5000,) (10,) 

In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
x = df[['marginal_cost', 'characteristic1', 'characteristic2']]
y = df['prices']

res = LinearRegression().fit(x, y)

print('Intercept: \n', res.intercept_)
print('Coefficients: \n', res.coef_)

predictions_price = res.predict(x)
print(predictions_price)
df['predict_prices'] = predictions_price
print(df)



Intercept: 
 -0.6366441737741262
Coefficients: 
 [1.4300757  0.32705265 0.13208319]
[22.43507478 18.91289503 20.66996847 20.59401513 23.77075862 22.50986118
 19.23015237 21.56115671 20.83967347 21.44425426 20.79195679 18.05951777
 19.32616293 19.01994938 22.46571348 21.28946795 17.95174249 20.48458394
 19.28053615 20.43233022 22.30130383 18.9435507  20.98204116 20.07180317
 23.95037609 22.54356635 18.57401754 21.92253666 20.84397646 21.56010999
 24.34686663 20.00321379 22.46979703 22.44749176 26.54442283 25.68438463
 21.25752322 23.00426481 22.56153334 23.74167213 22.94912059 19.06543357
 21.00457994 21.63748677 26.08848321 24.76533613 20.36081226 21.75780879
 21.8092485  21.41022852 18.44064413 16.48864964 17.6401025  17.99687353
 20.36240195 20.13737199 17.3958055  18.28114704 17.92883646 18.06930882
 21.85083639 18.48636621 22.33830815 20.78537449 24.76659008 24.12844458
 19.97466085 22.081176   21.23146518 21.47808438 20.76160927 17.85377321
 21.79142889 19.68456503 23.30656807 23.

In [33]:
# N = 500 
# firms  = 10
# time = 100 

# all_val_b0 = []
# all_val_b1 = []
# all_val_b2 = []
# all_val_mu = []
# all_val_omega = []

# for l in range(1, 20):
    
#     res = minimize(f, theta_0, args=(df, N, firms, time), method = 'Nelder-Mead')
#     optim_vec = res.x

#     all_val_b0.append(optim_vec[0])
#     all_val_b1.append(optim_vec[1])
#     all_val_b2.append(optim_vec[2])
#     all_val_mu.append(optim_vec[3])
#     all_val_omega.append(optim_vec[4])


# df_estimation_values = pd.DataFrame({'beta_0': all_val_b0,
#                                     'beta_1': all_val_b1,
#                                     'beta_2': all_val_b2, 
#                                     'mu': all_val_mu, 
#                                     'omega': all_val_omega, 
#                                     })
# df_estimation_values.to_csv(f'../data/first_estimation_market_inter_100', index=False)


KeyboardInterrupt: 

In [ ]:
print(df_estimation_values)

      beta_0    beta_1    beta_2        mu     omega
0   2.023374 -0.489820 -0.305029  0.518237  1.028462
1   2.055564 -0.512854 -0.288167  0.492266  1.026314
2   2.051028 -0.499914 -0.307267  0.500001  1.000249
3   2.017785 -0.504160 -0.302465  0.505155  1.033399
4   2.093635 -0.498735 -0.301054  0.500574  0.998048
5   1.843942 -0.522664 -0.326418  0.465671  1.067583
6   2.039810 -0.504466 -0.309574  0.485628  1.008863
7   1.994532 -0.522212 -0.301604  0.483624  1.030449
8   2.082030 -0.504558 -0.293747  0.496732  1.001629
9   2.016016 -0.495521 -0.305660  0.496311  1.026883
10  1.889141 -0.509478 -0.313419  0.509069  1.054910
11  1.956675 -0.522705 -0.315966  0.448703  1.031509
12  2.066856 -0.496990 -0.300188  0.511554  1.008428
13  2.048073 -0.505202 -0.303148  0.494320  1.027422
14  2.006608 -0.498820 -0.300991  0.516772  1.003304
15  2.107890 -0.519785 -0.298911  0.468237  1.038445
16  2.009199 -0.498358 -0.300472  0.503484  1.042100
17  2.042887 -0.509131 -0.306204  0.474573  1.